In [2]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium
import plotly.express as px

In [4]:
#get mrt data
mrt_df = pd.read_csv('../../data/mrt_station_final.csv')
print(mrt_df.head())
coords = mrt_df[['Latitude', 'Longitude']].values.tolist()
names = mrt_df['MRT.Name']
mrt_lat = mrt_df['Latitude']
mrt_lon = mrt_df['Longitude']
# run this to start OTP: java -Xmx6G -jar otp-2.5.0-shaded.jar --load C:\Users\wjlwi\OneDrive\Desktop\Github\Public-Transportation-In-Singapore\Backend\otp  --serve
#ethan: java -Xmx6G -jar otp-2.5.0-shaded.jar --load /Users/ethan/Documents/GitHub/Public-Transportation-In-Singapore/Backend/otp/  --serve

# Sample API request to the OTP isochrone endpoint
response = requests.get("http://localhost:8080/otp/traveltime/isochrone?batch=true&location=1.3521,103.895&time=2023-04-12T10:19:03%2B02:00&modes=WALK,TRANSIT&arriveBy=false&cutoff=30M17S")
# isochrone_geojson = response.json()

FileNotFoundError: [Errno 2] No such file or directory: '../data/mrt_station_final.csv'

Write a function for fetching isochrome. Configure params here

In [14]:
def fetch_isochrone(lat, lon, modes, cutoff="20M01S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "true",
        "location": f"{lat},{lon}",
        "time": "2023-06-01T18:00:00+02:00",
        "modes": modes,
        "arriveBy": "false",
        "cutoff": cutoff
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None

x = fetch_isochrone(1.293657725, 103.8550812, modes = "TRANSIT")
features = x['features'][0]['geometry']['coordinates'][0][0]
print(features)


[[103.861, 1.286332], [103.861622, 1.286649], [103.862799, 1.287064], [103.863292, 1.287825], [103.864598, 1.289076], [103.865409, 1.289624], [103.8653, 1.290326], [103.865128, 1.291423], [103.865524, 1.292348], [103.865741, 1.293221], [103.865497, 1.29412], [103.864598, 1.293448], [103.863638, 1.29406], [103.862799, 1.294406], [103.862571, 1.294793], [103.862529, 1.29502], [103.862281, 1.296301], [103.862068, 1.296819], [103.862264, 1.298082], [103.862268, 1.298617], [103.862559, 1.300176], [103.86242, 1.300416], [103.862235, 1.301651], [103.861, 1.302118], [103.860943, 1.302158], [103.8592, 1.302174], [103.859171, 1.302185], [103.859053, 1.302214], [103.857872, 1.302685], [103.857401, 1.30274], [103.85616, 1.302772], [103.855602, 1.302732], [103.853983, 1.302394], [103.853803, 1.302402], [103.85346, 1.302214], [103.852004, 1.301635], [103.851056, 1.301266], [103.850205, 1.301106], [103.84884, 1.30085], [103.848406, 1.302116], [103.847844, 1.300416], [103.846607, 1.299325], [103.84610

In [15]:
import plotly.graph_objs as go
import random

f = go.FigureWidget()
f.layout.hovermode = 'closest'
f.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2

# just some traces with random data points  
num_of_traces = 5
random.seed = 42
for i in range(num_of_traces):
    y = [random.random() + i / 2 for _ in range(100)]
    trace = go.Scatter(y=y, mode='lines', line={ 'width': default_linewidth })
    f.add_trace(trace)

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
    for i,_ in enumerate(f.data):
        f.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)
    print(points.trace_index)

# we need to add the on_click event to each trace separately       
for i in range(len(f.data) ):
    f.data[i].on_click(update_trace)

# let's show the figure 
f

FigureWidget({
    'data': [{'line': {'width': 2},
              'mode': 'lines',
              'type': 'scatter',
              'uid': '91718225-3a5d-49ad-9141-fb41c5c3b542',
              'y': [0.6792190976902961, 0.22137394669315436, 0.12346152510345199,
                    0.5574328542179786, 0.6749896748534419, 0.9558036596231871,
                    0.5539608429075888, 0.43232499728394613, 0.062204483133893484,
                    0.22922460633447217, 0.6156718550876745, 0.29343791304210454,
                    0.22381472200928676, 0.7337766624047912, 0.4091522042511243,
                    0.15267404588000122, 0.49511081758509257, 0.7068165965848623,
                    0.10539412017544691, 0.48872228564727527, 0.024304425120788675,
                    0.8308194102862716, 0.6968532570243694, 0.2717300241183833,
                    0.36659692570162583, 0.24468525385759998, 0.7642528519293306,
                    0.004681270799024806, 0.7217004916620968, 0.868621424652154,
       

In [6]:
import plotly.graph_objects as go


fig = go.Figure(go.Scattermapbox(lat=[36,37,38],
                                 lon=[26,27,28],
                                 mode='markers',
                                 marker={'size':9,'color':'purple'},
                                 selected=go.scattermapbox.Selected(marker = {"color":"red", "size":25})
                                 )
                )

fig.update_layout(clickmode='event+select',
                  mapbox = {'style': "stamen-terrain",
                            'center': {'lon': 27.0, 'lat': 37.0 },
                            'zoom': 6},
                  margin = dict(l=0, r=0, t=0, b=0)
                  )

fig.show()

In [7]:
from plotly.subplots import make_subplots
f = go.FigureWidget(make_subplots(rows=2, cols=1))

trace1 = go.Scatter(x=[3, 4, 5], y=[10, 11, 12], name='trae1')
trace2 = go.Scatter(x=[6, 7, 8], y=[13, 14, 15])

f.add_trace(trace1, row=1, col=1)
f.add_trace(trace2, row=2, col=1)

def print_coordinates(trace, points, selector):
    if points.point_inds:  # Ensure a point is clicked
        # Extract the coordinates of the clicked marker
        print(trace)
        clicked_lon = trace.x[1]
        clicked_lat = trace.y[1]
        # Print the coordinates
        print(f"Clicked marker coordinates: Lat {clicked_lat}, Lon {clicked_lon}")

for trace in f.data:
    trace.on_click(print_coordinates)
f

FigureWidget({
    'data': [{'name': 'trae1',
              'type': 'scatter',
              'uid': '065d1603-687b-4d4a-8ae5-bb653ddfdafb',
              'x': [3, 4, 5],
              'xaxis': 'x',
              'y': [10, 11, 12],
              'yaxis': 'y'},
             {'type': 'scatter',
              'uid': 'af87af3f-69e8-432f-b30c-b96635e6d176',
              'x': [6, 7, 8],
              'xaxis': 'x2',
              'y': [13, 14, 15],
              'yaxis': 'y2'}],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0]},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0]},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.425]}}
})

Scatter({
    'name': 'trae1',
    'uid': '065d1603-687b-4d4a-8ae5-bb653ddfdafb',
    'x': [3, 4, 5],
    'xaxis': 'x',
    'y': [10, 11, 12],
    'yaxis': 'y'
})
Clicked marker coordinates: Lat 11, Lon 4
Scatter({
    'name': 'trae1',
    'uid': '065d1603-687b-4d4a-8ae5-bb653ddfdafb',
    'x': [3, 4, 5],
    'xaxis': 'x',
    'y': [10, 11, 12],
    'yaxis': 'y'
})
Clicked marker coordinates: Lat 11, Lon 4


Sample to Edit for Hovering

In [8]:
import plotly.graph_objects as go
# Create a Plotly figure
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
default_linewidth = 2
highlighted_linewidth = 3
color_sequence = ['blue', 'green', 'red', 'orange', 'purple', 'yellow']

def print_coordinates(trace, points, selector):
    print("Hi")

# Fetch and add isochrones for each MRT station
for index, (lat, lon) in enumerate(coords):
    mrt_name = mrt_df[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['MRT.Name'].iloc[0]
    # Add isochrone as a trace
    color = color_sequence[index%len(color_sequence)]
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lon=[lon],
        lat=[lat],
        text = [mrt_name],
        marker=dict(size=10,color=color),
        selected=go.scattermapbox.Selected(marker = {"size":25}),
        unselected=go.scattermapbox.Unselected(marker = {"color":"grey", "size":5})
    ))
for trace in fig.data:
    trace.on_click(print_coordinates)
# Define layout for the map
fig.update_layout(
    clickmode='event+select',
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)
fig.show()

Static Map using go.Figure()

In [9]:
import plotly.graph_objects as go
# Create a Plotly figure
fig = go.Figure()
color_sequence = ['blue', 'green', 'red', 'orange', 'purple', 'yellow'] 
# Fetch and add isochrones for each MRT station
for index, (lat, lon) in enumerate(coords):
    isochrone_geojson = fetch_isochrone(lat, lon, modes="TRANSIT")
    mrt_name = mrt_df[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['MRT.Name'].iloc[0]
    if isochrone_geojson and 'features' in isochrone_geojson:
        # Access the first feature in the returned GeoJSON
        feature = isochrone_geojson['features'][0]
        if 'geometry' in feature:
            geometry = feature['geometry']
            if 'coordinates' in geometry:
                coordinates = geometry['coordinates'][0][0]
                # Assuming coordinates are in [lon, lat] format
                lon_coords = [coord[0] for coord in coordinates]
                lat_coords = [coord[1] for coord in coordinates]
                # Add isochrone as a trace
                color = color_sequence[index%len(color_sequence)]
                fig.add_trace(go.Scattermapbox(
                    mode="lines",
                    lon=lon_coords,
                    lat=lat_coords,
                    name=f"Isochrone {lat}, {lon}",
                    line=dict(width=1, color=color),
                    fill = "toself",
                    marker=dict(size=0),
                    text=[mrt_name],  # Set text for hover
                    hoverinfo='text'
                ))
                fig.add_trace(go.Scattermapbox(
                    mode="markers",
                    lon=[lon],
                    lat=[lat],
                    text=[mrt_name],
                    name=mrt_name,
                    marker=dict(size=10,color=color),
                    textposition="bottom center",
                ))


# Define layout for the map
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)

# Show the map
fig.show()

Visualise all isochrones

In [17]:

m = folium.Map(location=coords[0], zoom_start=12)

# Fetch and add isochrones for each MRT station
for lat, lon in coords:
    isochrone_geojson = fetch_isochrone(lat, lon, modes = "TRANSIT")
    if isochrone_geojson:
        folium.GeoJson(isochrone_geojson, name=f"Isochrone {lat}, {lon}").add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)
m

In [ ]:
import folium

m = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

for index, row in mrt_df.iterrows():
    # Creating a popup message with a brief description or link
    popup_message = f"MRT Station: {index}<br>"
    popup_message += "Cycling 30M Isochrone available."  # Example message; customize as needed
    
    # Create and add a marker to the map
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_message, parse_html=True, max_width=450)
    ).add_to(m)
m


